<a href="https://colab.research.google.com/github/mobindaz/Forward_Bot/blob/main/TelegramBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot nest_asyncio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
import logging
from telegram import Update, InlineKeyboardMarkup, InlineKeyboardButton, InputMediaPhoto
from telegram.ext import Application, MessageHandler, filters, CallbackContext, CommandHandler, ApplicationBuilder, CallbackQueryHandler
import nest_asyncio

# Apply nest_asyncio to handle the event loop issue in Jupyter notebooks
nest_asyncio.apply()

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

logger = logging.getLogger(__name__)

# Your bot's API token
API_TOKEN = '7467877753:AAGVGVEhJsvDNyhVgZLFcdALi0tqQObN6wo'

# Your channel ID
CHANNEL_ID = '@mobindazz'

# URL of the other bot (replace with your actual bot's URL or action)
ORDER_BOT_URL = 'https://t.me/your_order_bot'

# In-memory storage for images and product info
product_images = []
product_description = ''
product_price = ''

# Define command handlers
async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text(
        text="Please send up to 5 images.\n\n"
             "After sending all images, send the product description and price.\n"
             "We will show you a summary for confirmation before forwarding to the channel."
    )

async def handle_images(update: Update, context: CallbackContext) -> None:
    global product_images
    if len(product_images) < 5:
        product_images.append(update.message.photo[-1].file_id)
        await update.message.reply_text(f"Received {len(product_images)}/5 images.")
    else:
        await update.message.reply_text("You have already sent 5 images. Please send the product description and price.")

async def handle_text(update: Update, context: CallbackContext) -> None:
    global product_description, product_price
    text = update.message.text

    if not product_description:
        product_description = text.strip()
        await update.message.reply_text("Product description saved. Now send the product price.")
    elif not product_price:
        product_price = text.strip()
        await update.message.reply_text("Product price saved. Here is a summary of the information.")

        # Prepare the summary message
        summary_text = "Here is the summary of the product information:\n\n"

        if product_images:
            summary_text += "Images:\n"
            for _ in product_images:
                summary_text += "✔️ Image received\n"

        summary_text += f"Description: {product_description}\nPrice: {product_price}\n\n"

        keyboard = [
            [InlineKeyboardButton("Forward to Channel", callback_data='forward')],
            [InlineKeyboardButton("Cancel", callback_data='cancel')]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)

        await update.message.reply_text(summary_text, reply_markup=reply_markup)

async def handle_button(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    await query.answer()

    if query.data == 'forward':
        logger.info("Forwarding to channel.")
        await forward_product(update, context)
    elif query.data == 'cancel':
        await context.bot.send_message(chat_id=query.from_user.id, text="Forwarding cancelled.")
        # Reset stored values
        reset_product_info()

async def forward_product(update: Update, context: CallbackContext) -> None:
    global product_images, product_description, product_price

    if product_images and product_description and product_price:
        # Send images
        media_group = []
        for file_id in product_images:
            media_group.append(InputMediaPhoto(media=file_id))

        await context.bot.send_media_group(chat_id=CHANNEL_ID, media=media_group)

        # Send description and price
        keyboard = [
            [InlineKeyboardButton("Order", url=ORDER_BOT_URL)]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)

        await context.bot.send_message(
            chat_id=CHANNEL_ID,
            text=f"Product Description: {product_description}\nProduct Price: {product_price}",
            reply_markup=reply_markup
        )

        # Send success message to the user
        await context.bot.send_message(chat_id=update.effective_user.id, text="Product details have been successfully forwarded to the channel.")

        # Reset stored values
        reset_product_info()
    else:
        await context.bot.send_message(chat_id=update.effective_user.id, text="Incomplete product details. Please provide all required information.")

def reset_product_info():
    global product_images, product_description, product_price
    product_images = []
    product_description = ''
    product_price = ''

def main():
    """Start the bot."""
    # Create the Application and pass it your bot's token
    application = ApplicationBuilder().token(API_TOKEN).build()

    # Add handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.PHOTO, handle_images))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_text))
    application.add_handler(CallbackQueryHandler(handle_button))

    # Start the Bot
    application.run_polling()

if __name__ == '__main__':
    main()


RuntimeError: Cannot close a running event loop